In [1]:
import os
import sqlite3

In [2]:
'''
一共 3 個 DB
1. 存 NER 結果
2. 存 POS 結果
3. 存 RE 結果: 含網址、Email、Username、(行動、市內)電話、IP
'''

db_files = [
    "JudicialAnalysis_NER.db",
    "JudicialAnalysis_POS.db",
    "JudicialAnalysis_RE.db",
    "JudicialAnalysis_R_NER.db",
    "JudicialAnalysis_R_POS.db",
    "JudicialAnalysis_R_RE.db",
]

db_handler = dict()

for db_file in db_files:
    con_temp = sqlite3.connect(db_file)
    cur_temp = con_temp.cursor()

    db_handler.setdefault(db_file, dict())
    
    db_handler[db_file]['con'] = con_temp
    db_handler[db_file]['cur'] = cur_temp



In [3]:
# 統計所有的 Crime
crimes = dict()

for db_file, db_handkle in db_handler.items():
    if 'RE' in db_file:
        check_res = db_handkle['con'].execute(f"SELECT keywords FROM results WHERE keyword_type = 'crimes'")
        temp = check_res.fetchone()
        while temp:
            keys = temp[0].strip().split(',')
            for key in keys:
                crimes.setdefault(key, 0)
                crimes[key] += 1
            temp = check_res.fetchone()
    if 'NER' in db_file:
        check_res = db_handkle['con'].execute(f"SELECT keywords FROM results WHERE keyword_type = 'LAW'")
        temp = check_res.fetchone()
        while temp:
            keys = temp[0].strip().split(',')
            for key in keys:
                crimes.setdefault(key, 0)
                crimes[key] += 1
            temp = check_res.fetchone()
crimes_sort = sorted(crimes.items(), key =lambda x : x[1],reverse=True)
crimes_sort[:20] 

[('刑事訴訟法', 125043),
 ('刑法', 88124),
 ('毒品危害防制條例', 47964),
 ('第449條', 29951),
 ('洗錢防制法', 28894),
 ('第451條', 24892),
 ('詐欺', 18558),
 ('公共危險', 18243),
 ('竊盜', 14532),
 ('刑事訴訟法第159條', 11362),
 ('過失傷害', 11274),
 ('中華民國刑法第339條', 9339),
 ('刑事訴訟法第299條', 8612),
 ('被害人告訴人', 8508),
 ('陸海空軍刑法', 8122),
 ('第477條', 7950),
 ('施用毒品', 7913),
 ('於本', 7382),
 ('中華民國刑法第185條', 7348),
 ('第299', 6935)]

In [4]:
len(crimes_sort)

150036

In [5]:
'''
> 準備整理資料
[edge]
1. crime <-> id
2. crime <-> keyword (按月份存放)
3. filter crime count keyword
'''

con_crime_id = sqlite3.connect('Orgnize_Crime_ID.db')
cur_crime_id = con_crime_id.cursor()

con_crime_key = sqlite3.connect('Orgnize_Crime_Key.db')
cur_crime_key = con_crime_key.cursor()

In [6]:
# 創建 Table
new = 1
if new:
    cur_crime_id.execute("CREATE TABLE results(jid, crime, month)")
    con_crime_id.commit()

    cur_crime_key.execute("CREATE TABLE results(crime, keyword, month)")
    con_crime_key.commit()

In [7]:
# 整理 crime <-> id
for db_file, db_handkle in db_handler.items():
    if 'RE' in db_file:
        check_res = db_handkle['con'].execute(f"SELECT * FROM results WHERE keyword_type = 'crimes'")
        temp = check_res.fetchone()
        while temp:
            temp_jid = temp[0].strip()
            temp_month = temp[1].strip()[:7]
            keys = temp[3].strip().split(',')
            for key in keys:
                # jid, crime, month
                to_db_content = "INSERT INTO results VALUES(?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                cur_crime_id.execute(to_db_content, (temp_jid, key, temp_month))
            temp = check_res.fetchone()
        con_crime_id.commit()
    if 'NER' in db_file:
        check_res = db_handkle['con'].execute(f"SELECT * FROM results WHERE keyword_type = 'LAW'")
        temp = check_res.fetchone()
        while temp:
            temp_jid = temp[0].strip()
            temp_month = temp[1].strip()[:7]
            keys = temp[3].strip().split(',')
            for key in keys:
                # jid, crime, month
                to_db_content = "INSERT INTO results VALUES(?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                cur_crime_id.execute(to_db_content, (temp_jid, key, temp_month))
            temp = check_res.fetchone()
        con_crime_id.commit()
con_crime_id.commit()
con_crime_id.close()

In [9]:
# 讀取用再次開啟
con_crime_id = sqlite3.connect('Orgnize_Crime_ID.db')
cur_crime_id = con_crime_id.cursor()

In [11]:
# 先找到所有跟詐欺、洗錢有關的案件 jid
fraud_jids = dict()
fraud_keys = ['詐', '339', '洗錢']

check_res = cur_crime_id.execute(f"SELECT * FROM results")
temp = check_res.fetchone()
total_count = 0
while temp:
    # jid, crime, month
    temp_jid = temp[0].strip()
    temp_crime = temp[1].strip()
    temp_month = temp[2].strip()

    temp_match_count = 0
    for fk in fraud_keys:
        if fk in temp_crime:
            temp_match_count += 1
    if temp_match_count:
        fraud_jids.setdefault(temp_month, set())
        fraud_jids[temp_month].add(temp_jid)
        total_count += 1 
    
    temp = check_res.fetchone()
print(total_count)

86241


In [16]:
# 找詐欺的 keys
from IPython.display import clear_output

fraud_key_dicts = dict()
count = 0

for temp_month, month_jids in fraud_jids.items():
    temp_total = len(month_jids)
    #month_jids = list(month_jids)
    #month_jids.sort()
    for idx, mjid in enumerate(month_jids):
        clear_output()
        print(f'>>正在分析: {temp_month} 的資料')
        print(f'>>目前進度: {idx+1}/{temp_total}')
        print(f'>>累積數量: {count}')

        fraud_key_dicts.setdefault(temp_month, dict())
        
        for db_file, db_handkle in db_handler.items():
            check_res = db_handkle['con'].execute(f"SELECT jid, date, keyword_type, keywords FROM results WHERE jid='{mjid}'")
            rows = check_res.fetchall()
            for row in rows:
                date = row[1].strip()
                k_type = row[2].strip()
                key_words = row[3].strip().split(',')
                
                fraud_key_dicts[temp_month].setdefault(k_type, dict())
                
                for temp_key in key_words:
                    temp_key = temp_key.strip()
                    fraud_key_dicts[temp_month][k_type].setdefault(temp_key, 0)
                    fraud_key_dicts[temp_month][k_type][temp_key] += 1
                    count += 1

>>正在分析: 2023-10 的資料
>>目前進度: 1993/1993
>>累積數量: 23345044


In [19]:
# 將關鍵字存下來

import pandas as pd

columns = ['month', 'type', 'keyword', 'frequence']


for temp_month, keywords_data in fraud_key_dicts.items():
    rows = []
    for k_type, keywords in keywords_data.items():
        keywords_list = sorted(keywords.items(), key=lambda x:x[1], reverse=True)
        for (kl, freq) in keywords_list:
            row = [temp_month, k_type, kl, freq]
            rows.append(row)

    df = pd.DataFrame(rows, columns=columns)
    
    file = f'fraud_freq_keywords({temp_month}).csv'
    
    df.to_csv(file, index=False)

In [ ]:
con_Fraud.close()

In [30]:
# 將關鍵字按類型存成 xlsx
import pandas as pd

columns = ['關鍵字詞', '出現次數']

for k_type, keywords in fraud_key_dicts.items():
    rows = []
    
    keywords_list = sorted(keywords.items(), key=lambda x:x[1], reverse=True)
    for (kl, freq) in keywords_list:
        row = [kl, freq]
        rows.append(row)
        
    
    df = pd.DataFrame(rows, columns=columns)
    
    xlsx_file = f'fraud({k_type}).xlsx'
    df.to_excel(xlsx_file)